Following https://www.youtube.com/watch?v=s2QEhSpmhVA

In [16]:
import torch
from torchvision import transforms
from PIL import Image

In [17]:
model = torch.hub.load("hustvl/yolop", "yolop", pretrained=True)

Using cache found in C:\Users\asus/.cache\torch\hub\hustvl_yolop_main
C:\Users\asus/.cache\torch\hub\hustvl_yolop_main\hubconf.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feat

In [28]:
img = Image.open("./frames/vid_frame_0.jpg", )
img.show()
overlay_img = img.copy()

t = transforms.ToTensor()
resize_transform = transforms.Resize((640,640))
tensor_to_img = transforms.ToPILImage()


features = t(img)
features = resize_transform(features)
# tensor_to_img(features).show()

features = features.unsqueeze(0)

In [46]:
img = torch.randn(1, 3, 640, 640)
overlay_img = torch.randn(1, 3, 640, 640)
# overlay_img to pil image
overlay_img = tensor_to_img(overlay_img.squeeze(0))
# det_out, de_seg_out, ll_seg_out = model(features)
det_out, de_seg_out, ll_seg_out = model(img)


In [47]:
print(det_out)

[tensor([[[[[ 3.7759e-01,  4.7600e-01,  3.0969e+00,  1.0391e+00, -2.5104e+01,
             4.5270e+00],
           [-9.7187e-02,  9.2194e-01,  3.1161e+00,  1.0949e+00, -2.3448e+01,
             4.3641e+00],
           [ 4.2196e-01,  5.1838e-01,  4.0669e+00,  8.6834e-01, -2.5858e+01,
             4.3714e+00],
           ...,
           [ 1.0456e+00, -1.9376e+00,  2.7180e+00,  6.5909e-01, -2.1200e+01,
             4.2677e+00],
           [ 1.9080e+00,  1.2553e-01,  3.7227e+00,  1.5977e+00, -1.9935e+01,
             4.3471e+00],
           [ 1.7433e-01,  6.2798e-01,  2.2412e+00,  2.2026e+00, -2.0483e+01,
             4.0700e+00]],

          [[ 1.1222e+00, -1.0795e+00,  4.1271e+00,  1.0453e+00, -2.3596e+01,
             4.8097e+00],
           [ 2.5563e-01, -8.9124e-01,  3.1820e+00,  1.0275e+00, -2.8206e+01,
             4.3427e+00],
           [ 6.7780e-01, -4.1291e-01,  4.1544e+00,  8.1619e-01, -2.8258e+01,
             4.8974e+00],
           ...,
           [ 2.8792e+00,  1.4783e+00, 

In [48]:
print(det_out[0].shape)

torch.Size([1, 3, 80, 80, 6])


In [49]:
import numpy as np
import cv2

def show_seg_result(
    img,
    result,
    index,
    epoch,
    save_dir=None,
    is_ll=False,
    palette=None,
    is_demo=False,
    is_gt=False,
):
    # img = mmcv.imread(img)
    # img = img.copy()
    # seg = result[0]
    if palette is None:
        palette = np.random.randint(0, 255, size=(3, 3))
    palette[0] = [0, 0, 0]
    palette[1] = [0, 255, 0]
    palette[2] = [255, 0, 0]
    palette = np.array(palette)
    assert palette.shape[0] == 3  # len(classes)
    assert palette.shape[1] == 3
    assert len(palette.shape) == 2

    if not is_demo:
        color_seg = np.zeros((result.shape[0], result.shape[1], 3), dtype=np.uint8)
        for label, color in enumerate(palette):
            color_seg[result == label, :] = color
    else:
        color_area = np.zeros(
            (result[0].shape[0], result[0].shape[1], 3), dtype=np.uint8
        )

        # for label, color in enumerate(palette):
        #     color_area[result[0] == label, :] = color

        color_area[result[0] == 1] = [0, 255, 0]
        color_area[result[1] == 1] = [255, 0, 0]
        color_seg = color_area

    # convert to BGR
    color_seg = color_seg[..., ::-1]
    # print(color_seg.shape)
    color_mask = np.mean(color_seg, 2)
    img[color_mask != 0] = img[color_mask != 0] * 0.5 + color_seg[color_mask != 0] * 0.5
    # img = img * 0.5 + color_seg * 0.5
    img = img.astype(np.uint8)
    img = cv2.resize(img, (1280, 720), interpolation=cv2.INTER_LINEAR)

    if not is_demo:
        if not is_gt:
            if not is_ll:
                cv2.imwrite(
                    save_dir + "/batch_{}_{}_da_segresult.png".format(epoch, index), img
                )
            else:
                cv2.imwrite(
                    save_dir + "/batch_{}_{}_ll_segresult.png".format(epoch, index), img
                )
        else:
            if not is_ll:
                cv2.imwrite(
                    save_dir + "/batch_{}_{}_da_seg_gt.png".format(epoch, index), img
                )
            else:
                cv2.imwrite(
                    save_dir + "/batch_{}_{}_ll_seg_gt.png".format(epoch, index), img
                )
    return img


In [ ]:
# convert det_out to img
det_outn = det_out[0].cpu().detach().numpy()
det_outn = np.argmax(det_out, axis=0)

# convert de_seg_out to img
de_seg_outn = de_seg_out[0].cpu().detach().numpy()
de_seg_outn = np.argmax(de_seg_out, axis=0)

# convert ll_seg_out to img
ll_seg_outn = ll_seg_out[0].cpu().detach().numpy()
ll_seg_outn = np.argmax(ll_seg_out, axis=0)



AttributeError: 'numpy.ndarray' object has no attribute 'cpu'

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# from PIL import ImageDraw


# # Function to draw bounding boxes
# def draw_boxes(image, boxes, threshold=0.5):
#     draw = ImageDraw.Draw(image)
#     for box in boxes:
#         print(f"Box data: {box}")  # This will help you inspect the structure of `box`

#         # Adjust unpacking based on the actual structure of `box`
#         if len(box) == 6:
#             x_center, y_center, width, height, confidence, _ = box
#         elif len(box) == 7:  # Example of a case where there's one extra class ID
#             x_center, y_center, width, height, confidence, _, class_id = box
#         else:
#             print(f"Unexpected box structure: {box}")
#             continue

#         if confidence > threshold:
#             # Convert from center coordinates to top-left coordinates
#             x_min = int((x_center - width / 2) * image.width)
#             y_min = int((y_center - height / 2) * image.height)
#             x_max = int((x_center + width / 2) * image.width)
#             y_max = int((y_center + height / 2) * image.height)

#             # Draw the bounding box (optional: you can color-code it based on class_id)
#             draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=2)
#             draw.text((x_min, y_min), f"{confidence:.2f}", fill="white")

#     return image


# # Example usage:
# overlay_img = draw_boxes(overlay_img, det_out[0][0], threshold=0.5)
# overlay_img.show()


AttributeError: 'Tensor' object has no attribute 'load'

In [50]:
from PIL import Image, ImageDraw


def draw_road_on_black(image, detections, road_class_index, threshold=0.5):
    # Ensure image is a PIL object
    if not isinstance(image, Image.Image):
        raise ValueError("Provided image is not a valid PIL image")

    # Check if the image has a size attribute
    if not hasattr(image, "size") or len(image.size) != 2:
        raise ValueError("Image must have a valid size attribute (width, height)")

    # Create a black background image
    black_image = Image.new("RGB", image.size, (0, 0, 0))
    draw = ImageDraw.Draw(black_image)

    # Filter detections for the 'road' class
    for detection in detections:
        # Format: [x_center, y_center, width, height, confidence, class]
        x_center, y_center, width, height, confidence, class_id = detection

        if confidence > threshold and class_id == road_class_index:
            # Convert from center coordinates to top-left coordinates
            x_min = int((x_center - width / 2) * image.width)
            y_min = int((y_center - height / 2) * image.height)
            x_max = int((x_center + width / 2) * image.width)
            y_max = int((y_center + height / 2) * image.height)

            # Draw the bounding box on the black image
            draw.rectangle([x_min, y_min, x_max, y_max], outline="white", width=2)

    return black_image


# Example usage:
# Assuming det_out contains the YOLO model's output detections for the image.
# road_class_index is the index of the class "road" in your model (for example, let's assume it's 0).
road_class_index = 0  # Update with the correct class index for "road"
overlay_img = draw_road_on_black(
    overlay_img, det_out[0][0], road_class_index, threshold=0.5
)
overlay_img.show()  # Display the black image with detected roads


ValueError: too many values to unpack (expected 6)